In [95]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNorm2d
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV

In [96]:
num_classes = 10
img_rows, img_cols = 28, 28

In [97]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [98]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test .reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [99]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255

In [100]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [112]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):

    model = Sequential()
    model.add(Conv2D(filters, kernel_size,
                     padding='valid',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))    
    model.add(MaxPooling2D(pool_size=pool_size))    
    model.add(Dropout(0.5))
    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

In [113]:
dense_size_candidates = [(32), (64), (32, 32), (64, 64)]
my_classifier = KerasClassifier(make_model, batch_size=32)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(x_train, y_train)

print('The parameters of the best model are: ')
print(validator.best_params_)

Epoch 1/3
48000/48000 [==============================] - 32s 670us/step - loss: 0.7583 - accuracy: 0.7140
Epoch 2/3
48000/48000 [==============================] - 31s 648us/step - loss: 0.5912 - accuracy: 0.7772
Epoch 3/3
48000/48000 [==============================] - 31s 652us/step - loss: 0.5496 - accuracy: 0.7954
Epoch 1/3
48000/48000 [==============================] - 33s 688us/step - loss: 0.7766 - accuracy: 0.7111
Epoch 2/3
48000/48000 [==============================] - 35s 728us/step - loss: 0.5986 - accuracy: 0.7773
Epoch 3/3
48000/48000 [==============================] - 32s 669us/step - loss: 0.5471 - accuracy: 0.7992
Epoch 1/3
48000/48000 [==============================] - 32s 667us/step - loss: 0.7611 - accuracy: 0.7163
Epoch 2/3
48000/48000 [==============================] - 31s 648us/step - loss: 0.5739 - accuracy: 0.7875
Epoch 3/3
48000/48000 [==============================] - 31s 649us/step - loss: 0.5214 - accuracy: 0.8091
Epoch 1/3
48000/48000 [=======================

48000/48000 [==============================] - 34s 716us/step - loss: 0.4737 - accuracy: 0.8313
Epoch 1/6
48000/48000 [==============================] - 35s 731us/step - loss: 0.7747 - accuracy: 0.7078
Epoch 2/6
48000/48000 [==============================] - 37s 775us/step - loss: 0.5928 - accuracy: 0.7784
Epoch 3/6
48000/48000 [==============================] - 35s 739us/step - loss: 0.5397 - accuracy: 0.8003
Epoch 4/6
48000/48000 [==============================] - 35s 735us/step - loss: 0.5087 - accuracy: 0.8156
Epoch 5/6
48000/48000 [==============================] - 36s 742us/step - loss: 0.4899 - accuracy: 0.8238
Epoch 6/6
48000/48000 [==============================] - 34s 714us/step - loss: 0.4674 - accuracy: 0.8317
Epoch 1/6
48000/48000 [==============================] - 35s 733us/step - loss: 0.7734 - accuracy: 0.7069
Epoch 2/6
48000/48000 [==============================] - 33s 697us/step - loss: 0.6039 - accuracy: 0.7698
Epoch 3/6
48000/48000 [==============================] -

/Users/n.arai/opt/anaconda3/envs/gridsearch/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/n.arai/opt/anaconda3/envs/gridsearch/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/n.arai/opt/anaconda3/envs/gridsearch/lib/python3.7/site-packages/keras/wrappers/scikit_learn.py", line 209, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/Users/n.arai/opt/anaconda3/envs/gridsearch/lib/python

Epoch 1/6
60000/60000 [==============================] - 41s 678us/step - loss: 0.7448 - accuracy: 0.7164
Epoch 2/6
60000/60000 [==============================] - 39s 654us/step - loss: 0.5702 - accuracy: 0.7888
Epoch 3/6
60000/60000 [==============================] - 39s 655us/step - loss: 0.5281 - accuracy: 0.8059
Epoch 4/6
60000/60000 [==============================] - 39s 653us/step - loss: 0.4957 - accuracy: 0.8202
Epoch 5/6
60000/60000 [==============================] - 39s 655us/step - loss: 0.4705 - accuracy: 0.8283
Epoch 6/6
60000/60000 [==============================] - 39s 652us/step - loss: 0.4559 - accuracy: 0.8365
The parameters of the best model are: 
{'dense_layer_sizes': (32, 32), 'epochs': 6, 'filters': 8, 'kernel_size': 3, 'pool_size': 2}


In [114]:
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

10000/10000 [==============================] - 2s 200us/step
loss :  0.40840033097267153
accuracy :  0.8533999919891357
